# Pakiety i wczytanie danych 

In [7]:
#Import pakietów 
import os 
import numpy as np 
from PIL import Image 
import tensorflow as tf 
import warnings 
warnings.filterwarnings("ignore")

In [8]:
#Ścieżka do katalogu głównego
data_dir = "./clouds_confirm" 

In [17]:
def load_images_and_labels(data_dir): 
    images = [] 
    labels = [] 
    # przechodzenie przez każdy podkatalog w katalogu głównym
    for folder in os.listdir(data_dir): 
        folder_path = os.path.join(data_dir,folder) 
        if os.path.isdir(folder_path):  
            # przechodzenie przez każdy plik w podkatalogu 
            for file in os.listdir(folder_path): 
                file_path = os.path.join(folder_path, file) 
                # wczytanie obrazu 
                image = Image.open(file_path)
                #normalizacja do zakresu 0-1 
                image = np.array(image) / 255.0  
                images.append(image)
                #Przypisywanie etykiety na podstawie nazwy pliku 
                label = "not_cloud" if "not_cloud" in file else "cloud" 
                labels.append(label) 
    
    return np.array(images), np.array(labels)           

In [18]:
# Wczytywanie obrazów i etykiet 
images, labels = load_images_and_labels(data_dir) 

In [19]:
#Konwersja etyket na format numeryczny 
labels = np.array([0 if label == "not_cloud" else 1 for label in labels])

In [20]:
#Ile mamy wystąpień w każdej z klas? 
print(np.unique(labels,return_counts=True))

(array([0, 1]), array([ 2629, 44811]))


# Podział na dane treningowe, walidacyjne i testowe

In [22]:
from sklearn.model_selection import train_test_split 

# Najpierw dzielimy dane na część treningową i pozostałą 
train_images, temp_images, train_labels, temp_labels = train_test_split(
    images, labels, test_size=0.30, random_state=42
)
# Pozostałą część dzielimy na zbiory walidacyjny i testowy 
val_images, test_images, val_labels, test_labels = train_test_split(
    temp_images,temp_labels, test_size=0.50, random_state=42
)


In [23]:
print(f"Liczba obrazów treningowych: {len(train_images)}") 
print(f"Liczba obrazów walidacyjnych: {len(val_images)}")  
print(f"Liczba obrazów testowych: {len(test_images)}")

Liczba obrazów treningowych: 33208
Liczba obrazów walidacyjnych: 7116
Liczba obrazów testowych: 7116


# Model 1
## Bez balansu klasowego

In [24]:
from keras.api.models import Sequential 
from keras.api.layers import Conv2D, MaxPooling2D, Flatten, Dense
# Budowanie modelu
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [25]:

# Kompilacja modelu 
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [28]:
#Wyświetlenie struktury modelu 
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 98, 98, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 49, 49, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 47, 47, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 33856)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     2,166,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,185,729 (8.34 MB)

 Trainable params: 2,185,729 (8.34 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
# Dodanie callbacku do zapisywania
from keras.api.callbacks import ModelCheckpoint 

model_checkpoint_callback = ModelCheckpoint(
    filepath='model-{epoch:02d}-{val_accuracy:.2f}.keras',  # Nazwa pliku zapisu
    save_weights_only=False,  # Zapisuje pełny model, nie tylko wagi
    monitor='val_accuracy',  # Monitoruje dokładność na zbiorze walidacyjnym
    mode='max',  # Mode 'max' ponieważ celujemy w maksymalizację dokładności
    save_best_only=True,  # Zapisuje tylko, gdy dokładność jest lepsza niż dotychczas najlepsza
    verbose=1  # Wyświetla informacje o zapisie
)

In [31]:
history = model.fit(
    train_images, train_labels,
    validation_data=(val_images, val_labels),
    epochs=10,  # Liczba epok trenowania, można dostosować
    callbacks=[model_checkpoint_callback]  # Dodanie callbacku do listy callbacków
)

Epoch 1/10
1038/1038 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.9451 - loss: 0.1692
Epoch 1: val_accuracy improved from -inf to 0.95208, saving model to model-01-0.95.keras
1038/1038 ━━━━━━━━━━━━━━━━━━━━ 49s 47ms/step - accuracy: 0.9451 - loss: 0.1692 - val_accuracy: 0.9521 - val_loss: 0.1347
Epoch 2/10
1038/1038 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.9618 - loss: 0.1088
Epoch 2: val_accuracy improved from 0.95208 to 0.96585, saving model to model-02-0.97.keras
1038/1038 ━━━━━━━━━━━━━━━━━━━━ 48s 46ms/step - accuracy: 0.9618 - loss: 0.1088 - val_accuracy: 0.9659 - val_loss: 0.1101
Epoch 3/10
1038/1038 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.9756 - loss: 0.0763
Epoch 3: val_accuracy did not improve from 0.96585
1038/1038 ━━━━━━━━━━━━━━━━━━━━ 48s 46ms/step - accuracy: 0.9756 - loss: 0.0763 - val_accuracy: 0.9535 - val_loss: 0.1133
Epoch 4/10
1037/1038 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.9762 - loss: 0.0738
Epoch 4: val_accuracy improved from 0.96585 to 0.